# **Get the mean/average color of each frame and show it as a barcode**

## **ANLEITUNG**
### **[1. Voreinstellung]** Unter "Paths" (Zelle 4) beide Pfade anpassen.
### **[2. Voreinstellung]** In der letzten Zelle (Generate movie barcode) unter "walkthrough_name" den Namen des Walkthroughs anpassen.
### **[Mean/Average-Color aus jedem Frame bekommen]** Die Zelle bei "Get frame list" ausführen. Der aktuelle Stand wird fortlaufend angezeigt.
### **[Bei Abbruch, Verbindungsproblemen, usw.]** Die Zelle bei "Continue process [...]" ausführen. Der aktuelle Stand wird fortlaufend angezeigt.
### **[Wenn Prozess abgeschlossen]** Die Zelle bei "Generate the movie barcode" ausführen. Der MovieBarcode wird in verschiedenen Formaten generiert und abgespeichert.

## **Packages needed to be installed**

In [ ]:
!pip install colorthief

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## **Imports**

In [ ]:
from colorthief import ColorThief
from natsort import natsorted, ns
from google.colab.patches import cv2_imshow
# use next line if you want to clear output (right now not needed)
#from IPython.display import clear_output 

import matplotlib.pyplot as plt
import os
import skimage.io as io
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd

## **Connection to Google Drive**
### Doesn't have to be executed. Alternatively, the mounting can also be initialized manually in the left area.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Paths**
### Enter the path for the frames folder and the folder where the results should be saved.

In [ ]:
# folder in which the frames of the walkthrough are saved (Don't forget the "/" at the end)
path_to_game_frames = "/content/drive/MyDrive/digital_humanities/Walkthroughts/Siedler_Frames/"
# folder in which the results are to be saved (Don't forget the "/" at the end)
path_to_result_data = "/content/drive/MyDrive/digital_humanities/Ergebnisse/Color_Analysis/"

## **Get frame list**
### for each frame the mean/average color is generated and saved in an array 
### save the names of the frames that have already been analyzed (in case the process is aborted)
### show the current status of the process

In [ ]:
# get mean/average color for each frame and save it into the average_color_arr array

# list of frames in the folder
frames_list = os.listdir(path_to_game_frames)
# sorted list of frames in the folder
frames_list = natsorted(frames_list, alg=ns.PATH | ns.IGNORECASE)

# names of already analyzed frames are saved here. Serves to find out at which point the process should continue to run in the event of a process crash.
backup_check = []

mean_color_arr = []

for frame in frames_list:
  if frame.split(".")[-1].lower() in {"jpeg", "jpg"}:
    source_image = cv2.imread(path_to_game_frames + frame)
    average_color_row = np.average(source_image, axis=0)
    average_color = np.average(average_color_row, axis=0)

    average_color_tuple = tuple(average_color)

    mean_color_arr.append(average_color_tuple)
    backup_check.append(frame)

    # Shows how many frames have already been analyzed

    # use next line if you want to clear output (right now not needed)
    # clear_output()
    print(f'\rProcessing: {str(len(mean_color_arr))} of {str(len(frames_list))} Frames have been analyzed.', end='')

mean_color_arr = tuple(tuple(map(int, tup)) for tup in mean_color_arr) 

## **Continue process after disconnects, time ups, ...**

In [ ]:
# get dominant color for each frame and save it into the dominant_color_arr array

for frame in frames_list:
  if frame.split(".")[-1].lower() in {"jpeg", "jpg"}:
    if frame not in backup_check:
      source_image = cv2.imread(path_to_game_frames + frame)
      average_color_row = np.average(source_image, axis=0)
      average_color = np.average(average_color_row, axis=0)

      average_color_tuple = tuple(average_color)

      mean_color_arr.append(average_color_tuple)
      backup_check.append(frame)

      # Shows how many frames have already been analyzed

      # use next line if you want to clear output (right now not needed)
      # clear_output()
      print(f'\rProcessing: {str(len(mean_color_arr))} of {str(len(frames_list))} Frames have been analyzed.', end='')
      
    else: print(f'\rProcessing: searching for already analyzed frames', end='')

mean_color_arr = tuple(tuple(map(int, tup)) for tup in mean_color_arr) 

## **Generate the movie barcode**
### and saves as vector graphics (.eps, .svg) and .png and .jpg

In [ ]:
# generates the movie barcode

plt.imshow([[mean_color_arr[i] for i in range(len(mean_color_arr))]], extent=[0,len(mean_color_arr),0,1], aspect='auto')

# adjust here the name of the walkthrough
walkthrough_name = "test"

plt.savefig(path_to_result_data + walkthrough_name + '_average_color_for_each_frame.eps', format='eps')
plt.savefig(path_to_result_data + walkthrough_name + '_average_color_for_each_frame.svg', format='svg')
plt.savefig(path_to_result_data + walkthrough_name + '_average_color_for_each_frame.png')
plt.savefig(path_to_result_data + walkthrough_name + '_average_color_for_each_frame.jpg')